# Pydantic Lab: สร้างโมเดลข้อมูลขั้นเทพใน Python (ฉบับเข้าใจง่าย)

### **ภาพรวม**

แล็บนี้จะสอนวิธีการใช้ `BaseModel` และ `Field` ของ Pydantic สำหรับการตรวจสอบความถูกต้องของข้อมูล (Data Validation), การแปลงข้อมูล (Serialization) และการสร้างโมเดลข้อมูลที่แข็งแกร่งใน Python ซึ่ง Pydantic เป็นเครื่องมือที่จำเป็นอย่างยิ่งสำหรับแอปพลิเคชัน Python สมัยใหม่ โดยเฉพาะงานด้าน API และการประมวลผลข้อมูล

---

### **ส่วนที่ 1: เริ่มต้นกับ `BaseModel`**

`Pydantic BaseModel` คือ **คลาสหลัก** ที่ใช้ในไลบรารี Pydantic สำหรับ **สร้างโมเดลข้อมูลในภาษา Python**

โดยมีความสามารถหลักๆ คือ:
* ✅ **ตรวจสอบความถูกต้องของข้อมูล (Data Validation):** เช็คว่าข้อมูลที่รับเข้ามาตรงตามชนิดและเงื่อนไขที่เรากำหนดหรือไม่
* 🔄 **แปลงชนิดข้อมูลอัตโนมัติ (Parsing):** เช่น แปลงข้อความที่เป็นตัวเลข (`"30"`) ให้เป็นข้อมูลชนิดตัวเลข (`30`) โดยอัตโนมัติ

> **อธิบายง่ายๆ:** `BaseModel` คือแม่แบบสำหรับสร้างคลาสที่เราสามารถกำหนดชนิดของข้อมูล (Type Annotations) ได้อย่างชัดเจน และ Pydantic จะตรวจสอบข้อมูลให้ทันทีเมื่อเราสร้างอ็อบเจกต์จากคลาสนั้น

**ตัวอย่างการใช้งานเบื้องต้น:**

```python
from pydantic import BaseModel

class User(BaseModel):
    name: str
    age: int

user = User(name="Alice", age=30)        # ✅ ถูกต้อง ข้อมูลตรงตามที่กำหนด
user2 = User(name="Bob", age="30")       # ✅ ถูกต้อง Pydantic แปลง "30" เป็น int ให้
# user3 = User(name="Tom", age="abc")      # ❌ เกิด Error เพราะ "abc" ไม่สามารถแปลงเป็น int ได้
```

---

### **แบบฝึกหัด 1.1: สร้าง `BaseModel` แรกของคุณ**

ลองสร้างโมเดล `User` อย่างง่าย ที่มีข้อมูลชื่อ, อายุ, อีเมล และสถานะการใช้งาน

In [1]:
from pydantic import BaseModel
from typing import Optional

class User(BaseModel):
    name: str
    age: int
    email: str
    is_active: bool = True  # กำหนดค่าเริ่มต้นให้เป็น True

# สร้างอ็อบเจกต์จากโมเดล
user = User(name="Alice", age=25, email="alice@example.com")

# แสดงผลข้อมูล
print(user)

# เข้าถึงข้อมูลแต่ละฟิลด์
print(f"Name: {user.name}")

# แปลงโมเดลเป็น Dictionary (JSON)
print(f"Dictionary: {user.model_dump()}")

name='Alice' age=25 email='alice@example.com' is_active=True
Name: Alice
Dictionary: {'name': 'Alice', 'age': 25, 'email': 'alice@example.com', 'is_active': True}


### **แบบฝึกหัด 1.2: การตรวจสอบข้อมูลอัตโนมัติ**

ลองสร้างข้อมูลที่ไม่ถูกต้องและดูว่า Pydantic จัดการกับมันอย่างไร

In [2]:
# โค้ดนี้จะทำให้เกิด ValidationError
try:
    # พยายามสร้าง user ที่มี "age" ไม่ใช่ตัวเลข
    invalid_user = User(name="Bob", age="not a number", email="bob@example.com")
except Exception as e:
    print(f"Validation Error: {e}")

# โค้ดนี้จะทำงานได้ - Pydantic แปลง string "30" เป็น int ให้
valid_user = User(name="Charlie", age="30", email="charlie@example.com")
print(valid_user)
print(f"Age: {valid_user.age}, Type: {type(valid_user.age)}")

Validation Error: 1 validation error for User
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='not a number', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing
name='Charlie' age=30 email='charlie@example.com' is_active=True
Age: 30, Type: <class 'int'>


---

### **ส่วนที่ 2: รู้จักกับ `Field`**

`Field` คือฟังก์ชันที่ใช้ **กำหนดเงื่อนไขและรายละเอียดเพิ่มเติม** ให้กับแอตทริบิวต์ในโมเดลของเรา เช่น การกำหนดค่าเริ่มต้น, คำอธิบาย, หรือเงื่อนไขของข้อมูล (เช่น ตัวเลขต้องมากกว่า 0)

### **แบบฝึกหัด 2.1: การใช้งาน `Field` เบื้องต้น**

สร้างโมเดล `Product` ที่มีเงื่อนไขต่างๆ

**พารามิเตอร์ที่น่าสนใจของ `Field`:**

* `...` (Ellipsis): ใช้เพื่อบอกว่าฟิลด์นี้ **จำเป็น (Required)**
* `default`: กำหนดค่าเริ่มต้นถ้าไม่มีการส่งค่าเข้ามา
* `gt`, `ge`, `lt`, `le`: เงื่อนไขเชิงตัวเลข (มากกว่า, มากกว่าหรือเท่ากับ, น้อยกว่า, น้อยกว่าหรือเท่ากับ)
* `min_length`, `max_length`: กำหนดความยาวขั้นต่ำและสูงสุดของข้อความ
* `description`: เพิ่มคำอธิบายสำหรับฟิลด์นั้นๆ เพื่อสร้างเอกสารอ้างอิง

---

In [11]:
from pydantic import BaseModel, Field
import json

class Product(BaseModel):
    # Field(...) หมายถึงจำเป็นต้องระบุค่านี้
    name: str = Field(..., min_length=1, max_length=100, description="ชื่อสินค้า")
    price: float = Field(..., gt=0, description="ราคาต้องเป็นค่าบวก") # gt=0 คือ > 0

    # ge=0 คือ >= 0
    quantity: int = Field(default=0, ge=0, description="จำนวนสินค้าในสต็อก")
    category: str = Field("general", description="หมวดหมู่สินค้า")


# สร้างอ็อบเจกต์ product
product = Product(name="Laptop", price=999.99, quantity=5, category="electronics")
print(product)


# Alternative: Using json.dumps for even better formatting
product_dict = product.model_dump()
formatted_json = json.dumps(product_dict, indent=2, ensure_ascii=False)
print("\nFormatted JSON:")
print(formatted_json)

name='Laptop' price=999.99 quantity=5 category='electronics'

Formatted JSON:
{
  "name": "Laptop",
  "price": 999.99,
  "quantity": 5,
  "category": "electronics"
}



### **แบบฝึกหัด 3.: สร้างโมเดลสำหรับ API Response**

โค้ดนี้ใช้ Pydantic เพื่อสร้างโมเดลสำหรับการตอบกลับของ API (API Response) โดยกำหนดชนิดข้อมูลและเงื่อนไขของแต่ละฟิลด์อย่างชัดเจน เพื่อให้มั่นใจว่าโครงสร้างข้อมูลถูกต้องและปลอดภัย

In [13]:
from pydantic import BaseModel, Field
from typing import List, Optional
from datetime import datetime

class APIResponse(BaseModel):
    success: bool = Field(default=True)
    message: str
    data: Optional[dict] = None  # ข้อมูลอาจจะมีหรือไม่มีก็ได้
    timestamp: datetime = Field(default_factory=datetime.now) # สร้างเวลาปัจจุบันอัตโนมัติ
    errors: List[str] = Field(default_factory=list) # สร้าง list ว่างอัตโนมัติ

# ตัวอย่างการใช้งานเมื่อสำเร็จ
response = APIResponse(
    message="Data retrieved successfully",
    data={"users": [{"id": 1, "name": "Alice"}]}
)
print("Success response:")
print(response.model_dump_json(indent=2))



Success response:
{
  "success": true,
  "message": "Data retrieved successfully",
  "data": {
    "users": [
      {
        "id": 1,
        "name": "Alice"
      }
    ]
  },
  "timestamp": "2025-07-24T08:15:25.893692",
  "errors": []
}


### **แบบฝึกหัด 4: การสืบทอดคุณสมบัติ (Model Inheritance)**


In [17]:
from pydantic import BaseModel, Field  # นำเข้าเครื่องมือจาก Pydantic เพื่อสร้างโมเดล
from datetime import datetime  # นำเข้า datetime เพื่อจัดการวันที่และเวลา

# คลาสแม่: BaseEntity - คุณสมบัติพื้นฐานที่ทุกโมเดลจะสืบทอดมาใช้
class BaseEntity(BaseModel):
    id: int = Field(..., ge=1)  # ID ต้องเป็นจำนวนเต็ม และมากกว่าหรือเท่ากับ 1 (ge=greater or equal)
    created_at: datetime = Field(default_factory=datetime.utcnow)  # วันที่สร้างอัตโนมัติ (ใช้เวลา UTC ปัจจุบัน)
    updated_at: datetime = Field(default_factory=datetime.utcnow)  # วันที่อัปเดตอัตโนมัติ (เหมือน created_at)

# คลาสลูก: User - สืบทอดจาก BaseEntity (รับ id, created_at, updated_at มาฟรีๆ)
class User(BaseEntity):  # สืบทอดจาก BaseEntity
    name: str = Field(..., min_length=1)  # ชื่อต้องเป็นสตริง และยาวอย่างน้อย 1 ตัวอักษร
    email: str  # อีเมลเป็นสตริง (ไม่มีเงื่อนไขเพิ่มเติม)

# คลาสลูก: Product - สืบทอดจาก BaseEntity เหมือนกัน
class Product(BaseEntity):  # สืบทอดจาก BaseEntity
    name: str = Field(..., min_length=1)  # ชื่อสินค้าต้องยาวอย่างน้อย 1 ตัวอักษร
    price: float = Field(..., gt=0)  # ราคาต้องเป็นทศนิยม และมากกว่า 0 (gt=greater than)
    category: str  # หมวดหมู่เป็นสตริง

# การใช้งาน: สร้างอ็อบเจ็กต์ User และ Product
user = User(id=1, name="Alice", email="alice@example.com")  # สร้างผู้ใช้ Alice
product = Product(id=1, name="Laptop", price=999.99, category="Electronics")  # สร้างสินค้า Laptop


In [18]:

# แสดงผลในรูปแบบ JSON (เพื่อดูโครงสร้างข้อมูลชัดเจน)
print("User JSON:")
print(user.model_dump_json(indent=2))  # แปลง User เป็น JSON และเว้นบรรทัดสวยงาม
print("\nProduct JSON:")
print(product.model_dump_json(indent=2))  # แปลง Product เป็น JSON

User JSON:
{
  "id": 1,
  "created_at": "2025-07-24T01:35:37.544427",
  "updated_at": "2025-07-24T01:35:37.544429",
  "name": "Alice",
  "email": "alice@example.com"
}

Product JSON:
{
  "id": 1,
  "created_at": "2025-07-24T01:35:37.544453",
  "updated_at": "2025-07-24T01:35:37.544453",
  "name": "Laptop",
  "price": 999.99,
  "category": "Electronics"
}
